# Importing Libraries

In [240]:
import pandas as pd

# Defining Functions

In [241]:
def data_split(list_l):
    n = len(list_l[0])
    list1 = [list_l[0][:n//2],list_l[1][:n//2]]
    list2 = [list_l[0][n//2:],list_l[1][n//2:]] 
    #split input list into 2 halves list1 and list2
    return list1,list2

In [242]:
def data_clean(txt):
    temp_l = [] #creating a temporary list
    for i in range(len(txt)):
        temp_l.append(txt[i].replace(" ","").split(","))  #removing spaces and splitting by ',' and adding to temporary list

    year_list = []    #list to store all year values (2 from each line)
    temp_list = []     #list to store all temp values  (2 from each line)

    for i in range(len(temp_l)):
        #print(temp_l[i])          - each line has 2 Year values and 2 temp values 
        #print(temp_l[i][0][1:5])  - Year1
        #print(temp_l[i][1][:-1])  - Temp1
        #print(temp_l[i][2][1:5])  - Year2
        #print(temp_l[i][3][:-2])  - Temp2
        
        year_list.append([temp_l[i][0][1:5],temp_l[i][2][1:5]]) 
        temp_list.append([temp_l[i][1][:-1],temp_l[i][1][:-1]])
    
    year_temp_list = [year_list,temp_list] # a list with 1st element as year_list and 2nd as temp_list
    return year_temp_list

In [243]:
def mapper(year_temp_list):
    pair = [] #list to pair year and corresponding temp values
    for i in range(len(year_temp_list[0])):
        pair.append([year_temp_list[0][i][0], year_temp_list[1][i][0]]) #adding year-temp pair to paired list
        pair.append([year_temp_list[0][i][1], year_temp_list[1][i][1]]) #2nd year-temp pair, since 1 line has 2 pairs
    return pair

In [244]:
def reducer(pair_list):
    max_pair = [] #list to store year and max temperature
    max = 0
    year = pair_list[0][0] #take the first year value since it's sorted

    for i in pair_list:
        if i[0] == year:
            if int(i[1])>=max:
                max = int(i[1])  #if for the same year temp value is higher, 'max' is updated
            continue
        max_pair.append([year,max]) #append to list after getting max temp value for the previous year
        year = i[0] #update value of year to current Year
        max = int(i[1]) #update corresponding value of max temp for the current year
        
    max_pair.append([year,max]) #final year-temp pair is appended to list

    return max_pair #return list with pair of Year and it's maximum temperarure

In [245]:
def merge_sort(list1,list2):
    list1.extend(list2) #extend list1 with elements from list2 
    list1.sort() #sort combined list
    return list1 #return sorted list

In [246]:
def partition(merged_list):
    first_half = [] #list to store pairs where year<=2015
    second_half = [] #list to store pairs for years>2015
    for i in merged_list:
        if int(i[0])<2016:
            first_half.append(i) #year <= 2015
        else:
            second_half.append(i) #year >= 2016
    
    return first_half, second_half #return both halves

In [247]:
#The Main function which invokes other functions
def get_max_temp(txt):
    
    split1,split2 = data_split(data_clean(txt)) #get 2 splits of cleaned data

    map1 = mapper(split1) 
    map2 = mapper(split2)  #generate key value pair for both splits of data

    sorted_list = merge_sort(map1,map2)
    hf1,hf2 = partition(sorted_list)  #sorting the key value pairs and partitioning based of "Year<=2015"

    hf1_max = reducer(hf1) #reducer output of first half (year <= 2015)
    hf2_max = reducer(hf2) #reducer output of second half (year>2015)

    hf1_max.extend(hf2_max) #extending first half with second half, so after this hf1 will contain all reduced pairs

    df = pd.DataFrame({"Year":[i[0] for i in hf1_max],
                       "Max_Temp":[i[1] for i in hf1_max]}) #creating dataframe with year and it's max temp
    df.to_csv('output.csv',index=False) #writting DF to CSV file

# Reading File and Calling Main Function

In [248]:
input_path = 'C:/Users/rshara4/Downloads/temperatures.txt'
ifile = open(input_path,'r')
txt = ifile.readlines()
get_max_temp(txt)